# Initialisation

In [ ]:
val currentDirectory = new java.io.File(".").getCanonicalPath
val path = java.nio.file.FileSystems.getDefault().getPath(s"$currentDirectory/lib/MuLOT-local-assembly-0.4.jar")
val x = ammonite.ops.Path(path)
interp.load.cp(x)

In [ ]:
import mulot.local.Tensor
import mulot.local.tensordecomposition.cp.ALS

# Data loading

In [ ]:
import java.io.File

val tensorsFiles = new File("sample_tensors").listFiles
    .map(_.getName)
    .filter(f => f.startsWith("tensor") && !f.contains("clusters")).toList

In [ ]:
import scala.io.Source

def readFile(file: String): Map[Array[_], Double] = {
    
    val startTime = System.currentTimeMillis()
    
    val source = Source.fromFile(file)
    val r = (for (_entry <- source.getLines().drop(1)) yield {
        val entry = _entry.split(",")
        val key = Array[Int](entry(0).toInt, entry(1).toInt, entry(2).toInt)
        key -> entry(3).toDouble
    }).toMap
    source.close()
    val endTime = (System.currentTimeMillis() - startTime).toInt

    println("Reading time: " + (endTime / 1000) + "s")

    r.asInstanceOf[Map[Array[_], Double]]
}

# Run CP

In [ ]:
import scala.collection.mutable.{Map => MMap}

In [ ]:
// Remove logging
import $ivy.`com.outr::scribe:3.10.4`

scribe.Logger.root
  .clearHandlers()
  .clearModifiers()
  .withHandler(minimumLevel = Some(scribe.Level.Error))
  .replace()

In [ ]:
var timeMuLOTLocal = MMap[Int, MMap[Int, MMap[Double, Int]]]()

for (dimension <- 3 to 3;
     size <- List(100, 1000, 10000, 100000);
     sparsity <- List(1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10).reverse) {
    val fileName = s"tensor_${dimension}_${size}_${sparsity}.csv"
    if (tensorsFiles.contains(fileName)) {
        println(fileName)
        val nbIterations = 1
        var endTime = 0
        val tensor = Tensor(readFile(s"sample_tensors/$fileName"), dimension, Array("d0", "d1", "d2"))
        println("File loaded")
        for (j <- 0 until nbIterations) {
            
            val startTime = System.currentTimeMillis()

            val kruskal = ALS(tensor, 3).withMaxIterations(5).withMinFms(1.0).execute()

            endTime += (System.currentTimeMillis() - startTime).toInt

            println("Execution time: " + (endTime / 1000) + "s")
        }
        val finalTime = (endTime / nbIterations).toInt
        var dimMap = timeMuLOTLocal.getOrElse(dimension, MMap[Int, MMap[Double, Int]]())
        var sizeMap = dimMap.getOrElse(size, MMap[Double, Int]())
        sizeMap = sizeMap + (sparsity -> finalTime)
        dimMap = dimMap + (size -> sizeMap)
        timeMuLOTLocal(dimension) = dimMap
    } 
}

# Results

## Export results in CSV

In [ ]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.6`
import com.github.tototoshi.csv._

In [ ]:
val f = new java.io.File(s"""results/timeMuLOT-local.csv""")
val fileExists = f.exists()
val writer = CSVWriter.open(f, append = true)
if (!fileExists) {
    writer.writeRow(List[String]("dimension", "size", "sparsity", "time"))
}
for ((dimension, r1) <- timeMuLOTLocal; (size, r2) <- r1; (sparsity, time) <- r2) {
    println(List[Any](dimension, size, sparsity, time))
    writer.writeRow(List[Any](dimension, size, sparsity, time))
}
writer.close()